In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install transformers datasets torch scikit-learn streamlit
%pip install tf-keras
%pip install pandas

In [3]:
# Emotion Classification with BERT
from transformers import pipeline

classifier = pipeline("text-classification", model="bhadresh-savani/bert-base-uncased-emotion")


Device set to use cpu


In [5]:
text = "I'm feeling extremely anxious and can't sleep."
result = classifier(text)
print(result)

[{'label': 'fear', 'score': 0.9969569444656372}]


In [ ]:
# State management for dialogue systems
class DialogueStateTracker:
    def __init__(self):
        self.state = {
            "slots": {},
            "intents": [],
            "emotions": [],
            "history": []
        }

    def update(self, user_input, intent, slots, emotion):
        self.state["history"].append(user_input)
        self.state["intents"].append(intent)
        self.state["emotions"].append(emotion)
        for slot, value in slots.items():
            self.state["slots"][slot] = value

    def get_state(self):
        return self.state


Creating dataset for intents classification

In [35]:
import pandas as pd
import json

# Load the JSON file
with open('../dataset/data_full.json', 'r') as f:
    data = json.load(f)
    
# Create DataFrame from the list of lists in 'oos_val'
df_train_raw = pd.DataFrame(data['train'], columns=['query', 'label'])

# Save the DataFrame to a CSV file

df_train_raw.to_csv('data_full.csv', index=False)
df_train_raw

,query,label
0,what expression would i use to say i love you ...,translate
1,can you tell me how to say 'i do not speak muc...,translate
2,"what is the equivalent of, 'life is good' in f...",translate
3,"tell me how to say, 'it is a beautiful morning...",translate
4,"if i were mongolian, how would i say that i am...",translate
...,...,...
14995,can you explain why my card was declined,card_declined
14996,how come starbucks declined my card when i tri...,card_declined
14997,how come my card was not accepted yesterday,card_declined
14998,find out what happened to make my card get dec...,card_declined


In [36]:
whomadeyou_sample = df_train_raw.loc[df_train_raw['label'] == 'who_made_you'].sample(10, random_state=1)
whatcaniaskyou_sample = df_train_raw.loc[df_train_raw['label'] == 'what_can_i_ask_you'].sample(10, random_state=1)
whatsong_sample = df_train_raw.loc[df_train_raw['label'] == 'what_song'].sample(10, random_state=1)
whatareyourhob_sample = df_train_raw.loc[df_train_raw['label'] == 'what_are_your_hobbies'].sample(10, random_state=1)
doyouhavepets_sample = df_train_raw.loc[df_train_raw['label'] == 'do_you_have_pets'].sample(10, random_state=1)




whatcaniaskyou_sample.reset_index(drop=True, inplace=True)
whomadeyou_sample.reset_index(drop=True, inplace=True)
doyouhavepets_sample.reset_index(drop=True, inplace=True)
whatareyourhob_sample.reset_index(drop=True, inplace=True)
whatsong_sample.reset_index(drop=True, inplace=True)

merged_df = pd.concat([whomadeyou_sample, whatcaniaskyou_sample, whatareyourhob_sample, whatsong_sample, doyouhavepets_sample], ignore_index=True)
merged_df.loc[:,'label'] = 'smalltalk'
merged_df


,query,label
0,tell me what company programmed you,smalltalk
1,can you tell me who made the ai,smalltalk
2,who's responsible for your existence,smalltalk
3,"the creator of your ai, what is their name",smalltalk
4,i'd like to know who made you,smalltalk
5,i want to know who made you,smalltalk
6,what company's software engineers gave you life,smalltalk
7,who made the ai,smalltalk
8,who was your programmer,smalltalk
9,who is your originator,smalltalk


In [ ]:
basic_intents = ['greeting', 'goodbye', 'smalltalk']

In [ ]:
df_train = df_train_raw.loc[df_train_raw['label'].isin([]) 'smalltalk']

In [ ]:
smalltalk = ['what_can_i_ask_you',         # curiosity about bot capability
    'who_made_you',               # casual question about bot's origin
    'do_you_have_pets',           # friendly chit-chat
    'who_do_you_work_for',        # bot-related small talk
    'what_song',                  # general curiosity, possibly non-task
    'what_is_your_name',          # bot identification
    'what_are_your_hobbies',
    'how_old_are_you']
df_train.loc[df_train['label'].isin(smalltalk), 'label'] = 'smalltalk'
df_train.loc[df_train['label'] == 'smalltalk']

,query,label
2100,what things can you help me with exactly,smalltalk
2101,what are the things you can help me with,smalltalk
2102,what exactly can you help me with,smalltalk
2103,what can you help me with exactly,smalltalk
2104,what can you help me with,smalltalk
...,...,...
14095,what is your birthday,smalltalk
14096,tell me your date of birth,smalltalk
14097,tell me your age please,smalltalk
14098,tell me your birthday,smalltalk


In [29]:
df_train.loc[df['label'] == 'do_you_have_pets', :]

,query,label
8200,what type of pet do you have,do_you_have_pets
8201,share some information about your pets with me,do_you_have_pets
8202,do you have any type of pet,do_you_have_pets
8203,do you own pets,do_you_have_pets
8204,what are the pets that you have in your posses...,do_you_have_pets
...,...,...
8295,i want to know whether or not you are a pet owner,do_you_have_pets
8296,what sorts of pets do you own,do_you_have_pets
8297,i want to hear about your pets,do_you_have_pets
8298,are you a pet owner,do_you_have_pets


In [19]:
df_train_test = df_train.loc[df_train['label'].isin(['greeting', 'goodbye', 'smalltalk', 'thankyou'])]
df_train_test

,query,label
2100,what things can you help me with exactly,smalltalk
2101,what are the things you can help me with,smalltalk
2102,what exactly can you help me with,smalltalk
2103,what can you help me with exactly,smalltalk
2104,what can you help me with,smalltalk
...,...,...
14095,what is your birthday,smalltalk
14096,tell me your date of birth,smalltalk
14097,tell me your age please,smalltalk
14098,tell me your birthday,smalltalk
